In [1]:
# %% 1. 匯入套件與環境設定
import os, json, ast, time, math
import pandas as pd
import numpy as np
import openai, tiktoken
from PyPDF2 import PdfReader
from tqdm.auto import tqdm
from dotenv import load_dotenv
import matplotlib.pyplot as plt

load_dotenv()                               # 讀取 .env
openai.api_key = os.getenv("OPENAI_API_KEY")

PDF_DIR          = "data/tcfd_report_pdf_preprocessed"     # 54 份 PDF
LABEL_XLSX       = "data/tcfd第四層揭露指引.xlsx"
GROUND_TRUTH_XLSX= "data/answer/rank.xlsx"
OUT_DIR          = "data/llm_question_answering_results"
os.makedirs(OUT_DIR, exist_ok=True)

MODEL_NAME       = "gpt-4o-mini"
TOKENIZER        = tiktoken.encoding_for_model(MODEL_NAME)
CHUNK_TOKENS     = 1500          # 單 chunk token 上限（可依需要調整）
OVERLAP_TOKENS   = 150           # chunk 重疊
MAX_WORKERS      = 8             # 并行執行緒數量（依 GPU/CPU 調）
SAVE_INTERVAL    = 100           # 每處理 n 個 chunk 即落地一次 CSV


In [2]:
# %% 2. 讀取「第四層揭露指引」標籤
df_labels       = pd.read_excel(LABEL_XLSX).dropna(subset=["Label", "Definition"])
label_mapping   = dict(zip(df_labels["Label"], df_labels["Definition"]))
all_label_list  = list(label_mapping.keys())        
print(f"共載入 {len(all_label_list)} 個標籤：", all_label_list[:10], "...")


共載入 91 個標籤： ['G-1-1_1', 'G-1-1_2', 'G-1-2_3', 'G-1-2_4', 'G-1-3_5', 'G-2-1_6', 'G-2-1_7', 'G-2-1_8', 'G-2-2_9', 'G-2-2_10'] ...


In [3]:
# %% 3. PDF ➜ Text ➜ Chunk  (用 GPT token 長度切，附詳細 log)
def pdf_to_chunks(pdf_path: str,
                  max_tokens: int = CHUNK_TOKENS,
                  overlap_tokens: int = OVERLAP_TOKENS):
    """
    讀取 PDF，依 GPT token 長度切 chunk。
    - 先把全文一次 token 化（最快），再依 max_tokens/overlap_tokens 切。
    - 進入函式時會印：頁數、總 token 數、預估 chunk 數。
    """
    try:
        reader = PdfReader(pdf_path)
        pages  = len(reader.pages)
        full_text = "\n".join(p.extract_text() or "" for p in reader.pages)
    except Exception as e:
        print("❌ 解析失敗:", pdf_path, e, flush=True)
        return

    # 全文→token
    tokens   = TOKENIZER.encode(full_text, disallowed_special=())
    total_tok= len(tokens)

    # 預估 chunk 數量
    if total_tok <= max_tokens:
        est_chunks = 1
    else:
        step = max_tokens - overlap_tokens
        est_chunks = math.ceil((total_tok - max_tokens) / step) + 1

    print(f"  📄 {os.path.basename(pdf_path)} | 頁數={pages} | tokens={total_tok} | 估計 chunks={est_chunks}",
          flush=True)

    # 切段
    start   = 0
    chunk_id= 0
    while start < total_tok:
        end          = min(start + max_tokens, total_tok)
        chunk_tokens = tokens[start:end]
        chunk_text   = TOKENIZER.decode(chunk_tokens)
        yield chunk_id, chunk_text

        chunk_id += 1
        start = end - overlap_tokens  # 保持 overlap


In [4]:
# %% 4. 定義 LLM 查詢函式（prompt 100% 按原樣）
def build_matched_label_json(label_list):
    return json.dumps(
        [{"label": lb, "label_definition": label_mapping.get(lb, "")}
         for lb in label_list],
        ensure_ascii=False,
    )

ALL_LABEL_JSON = build_matched_label_json(all_label_list)   # ← 請確認 len(...) 是否 91
MAX_RETRY = 5                                               # 最多重試 5 次

def query_llm_for_verification(chunk: str):
    # ======= 這段 prompt 與你原始程式碼完全一致 =======
    prompt = f"""
        ### 背景資訊 ###
        你是氣候相關財務揭露標準專家，熟悉 TCFD 第四層揭露指引的標準與定義。你將會收到一段報告書內容，並且需要判斷該報告書內容是否符合某些特定揭露標準。

        請根據以下報告書內容進行判斷：
        {chunk}

        請僅針對以下揭露標準進行評估，不要評估或包含其他任何標準：
        {ALL_LABEL_JSON}

        ### 回覆格式 ###
        請僅回覆純 JSON 格式，不要包含任何 Markdown 語法、程式碼區塊或額外說明文字。每個 JSON 物件必須包含以下欄位：
        1. chunk: string，報告書內容。
        2. label: string，對應的揭露標準代碼。
        3. reason: string，詳細說明判斷的推理過程，解釋為何該揭露標準有或沒有被揭露。
        4. is_disclosed: boolean，若報告書中有揭露該標準則回覆 1；未揭露則回覆 0。

        請僅針對我提供的標準列表中的標準提供評估，不要添加任何其他標準。你的回覆應該包含同樣數量的 JSON 物件，每個對應到我提供的一個標準。

        [{', '.join([f'"{lb}"' for lb in all_label_list])}]
    """
    # ===============================================

    in_tok = len(TOKENIZER.encode(prompt))
    for retry in range(MAX_RETRY):
        try:
            start_t = time.time()
            resp = openai.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "你是氣候相關財務揭露標準專家。"},
                    {"role": "user",   "content": prompt}
                ],
                timeout=300,   # 300 秒
            )
            latency = time.time() - start_t
            # 如果 OpenAI 回傳 usage，就讀出來；否則顯示 '?'
            out_tok = (resp.usage.total_tokens - in_tok) if getattr(resp, "usage", None) else "?"
            print(f"      ↪ LLM ok  | retry={retry} | in_tok={in_tok} | out_tok={out_tok} | "
                  f"latency={latency:.1f}s", flush=True)
            return resp.choices[0].message.content.strip(), latency
        except Exception as e:
            wait = (2 ** retry) + np.random.rand()
            print(f"⚠️  LLM retry {retry+1}/{MAX_RETRY} after {wait:.1f}s → {e}", flush=True)
            time.sleep(wait)
    return "Error", None


In [ ]:
# %% 5. 逐檔案處理並即時寫檔（完整詳細 progress）
output_rows  = []
csv_path_tmp = os.path.join(OUT_DIR, "llm_raw_responses_tmp.csv")

pdf_files = sorted([f for f in os.listdir(PDF_DIR) if f.lower().endswith(".pdf")])
print("即將處理 PDF 數量：", len(pdf_files))

global_chunk_idx = 0

for pdf_idx, pdf_name in enumerate(tqdm(pdf_files, desc="PDF 檔案")):
    pdf_path = os.path.join(PDF_DIR, pdf_name)

    # **邊產生邊處理**，不用 list() 先全部載入
    
    for chunk_id, chunk_text in pdf_to_chunks(pdf_path):
        tk_len = len(TOKENIZER.encode(chunk_text))
        print(f"  ▶ [{pdf_idx+1}/{len(pdf_files)}] {pdf_name} | local_chunk={chunk_id} "
            f"global={global_chunk_idx} | len_tok={tk_len}", flush=True)

        # ====== 加入時間戳 ======
        print(f"      ⏳ call LLM {time.strftime('%H:%M:%S')} …", flush=True)
        response, latency = query_llm_for_verification(chunk_text)
        print(f"      ✅ done {time.strftime('%H:%M:%S')}  latency={latency:.1f}s", flush=True)
        # ========================

        if latency is None:
            print("    ✘ LLM 回傳 Error", flush=True)

        output_rows.append(
            {
                "Filename": pdf_name,
                "Chunk_ID": chunk_id,
                "Chunk_Text": chunk_text,
                "LLM_Response": response,
            }
        )
        global_chunk_idx += 1

        # 定期 flush
        if global_chunk_idx % SAVE_INTERVAL == 0:
            pd.DataFrame(output_rows).to_csv(
                csv_path_tmp, mode="a", index=False,
                header=not os.path.exists(csv_path_tmp),
                encoding="utf-8",
            )
            print(f"💾 flush @ {global_chunk_idx} chunks → {csv_path_tmp}", flush=True)
            output_rows.clear()

    

    print(f"✅ {pdf_name} 完成 | 累計 chunks={global_chunk_idx}", flush=True)

# 最後 flush
if output_rows:
    pd.DataFrame(output_rows).to_csv(
        csv_path_tmp, mode="a", index=False,
        header=not os.path.exists(csv_path_tmp), encoding="utf-8"
    )
    print(f"💾 最終 flush，總 chunks={global_chunk_idx}", flush=True)

print("🎉 全部 PDF 處理完成！CSV 位置：", csv_path_tmp)


即將處理 PDF 數量： 54


PDF 檔案:   0%|          | 0/54 [00:00<?, ?it/s]

  📄 上海商銀_2022.pdf | 頁數=47 | tokens=25412 | 估計 chunks=19
  ▶ [1/54] 上海商銀_2022.pdf | local_chunk=0 global=0 | len_tok=1500
      ↪ LLM ok  | retry=0 | in_tok=7043 | out_tok=1596 | latency=28.4s
  ▶ [1/54] 上海商銀_2022.pdf | local_chunk=1 global=1 | len_tok=1500


In [ ]:
# %% 6. 解析 LLM 回傳 JSON → 展開為 (Filename, Chunk_ID, Label, is_disclosed)
def parse_llm_json(resp_str):
    try:
        return ast.literal_eval(resp_str)
    except Exception as e:
        print("❌ 回傳格式解析失敗：", e)
        return []

raw_df        = pd.read_csv(csv_path_tmp)
raw_df["Parsed"] = raw_df["LLM_Response"].apply(parse_llm_json)
exploded_df   = raw_df.explode("Parsed").reset_index(drop=True)

exploded_df["Label"]        = exploded_df["Parsed"].apply(
    lambda x: x.get("label") if isinstance(x, dict) else None
)
exploded_df["is_disclosed"] = exploded_df["Parsed"].apply(
    lambda x: x.get("is_disclosed") if isinstance(x, dict) else None
)
exploded_df = exploded_df[
    ["Filename", "Chunk_ID", "Label", "is_disclosed"]
]

print("展開後資料筆數：", len(exploded_df))
display(exploded_df.head(5))


In [ ]:
# %% 7. 轉換 ➜ Institution / Year / Pivot Table
def extract_institution(fn):
    base = fn.split('.')[0]
    return base.split('_')[0]

def extract_year(fn):
    base = fn.split('.')[0]
    parts = base.split('_')
    return parts[1] if len(parts) > 1 else None

exploded_df["Institution"]   = exploded_df["Filename"].map(extract_institution)
exploded_df["Year"]          = exploded_df["Filename"].map(extract_year)
exploded_df["Answer"]        = exploded_df["is_disclosed"].map(lambda x: "Y" if x==1 else "N")

pivot_df = (
    exploded_df
    .pivot_table(index=["Institution", "Year"], columns="Label",
                 values="Answer", aggfunc="first")
    .reset_index()
)
pivot_path = os.path.join(OUT_DIR, "pdf_direct_llm_pivot.csv")
pivot_df.to_csv(pivot_path, index=False, encoding="utf-8")
print("已產生 pivot CSV：", pivot_path)
display(pivot_df.head(3))


In [ ]:
# %% 8. 與 Ground Truth 比對
answer_df = pd.read_excel(GROUND_TRUTH_XLSX)
answer_df.columns = answer_df.columns.astype(str)
pivot_df.columns  = pivot_df.columns.astype(str)

# key → answer row
ans_lookup = {
    (str(r["Financial_Institutions"]), str(r["Year"])): r
    for _, r in answer_df.iterrows()
}

common_q = [c for c in pivot_df.columns
            if c not in ("Institution", "Year") and c in answer_df.columns]

print(f"共同比對欄位：{len(common_q)}")

total, correct = 0, 0
per_inst = {}
errors   = {}

for _, row in pivot_df.iterrows():
    inst  = str(row["Institution"])
    year  = str(row["Year"])
    key   = (inst if inst.endswith("金") else inst, year)   # 依你原邏輯微調
    if key not in ans_lookup: 
        continue
    ans_row = ans_lookup[key]
    for q in common_q:
        pred = row[q]
        truth= ans_row[q]
        if pd.isna(truth): 
            continue
        total += 1
        per_inst.setdefault(inst, {"correct":0,"total":0})
        per_inst[inst]["total"] += 1
        if pred == truth:
            correct += 1
            per_inst[inst]["correct"] += 1
        else:
            errors.setdefault(inst, []).append(
                {"Year": year, "Question": q,
                 "Pred": pred, "Truth": truth}
            )

print(f"全體準確率：{correct}/{total} = {correct/total:.3%}")
for inst, cnt in per_inst.items():
    acc = cnt["correct"]/cnt["total"] if cnt["total"] else np.nan
    print(f"{inst}: {acc:.3%}")

# 顯示錯誤樣例
for inst, err_list in errors.items():
    if err_list:
        print(f"\n{inst} 有 {len(err_list)} 個不一致樣例：")
        for e in err_list[:5]:
            print(e)


In [ ]:
# %% 9. 各題目（Label）正確率長條圖
label_acc = (
    exploded_df.groupby("Label")["is_disclosed"]
    .apply(lambda x: (x==1).sum()/len(x))
    .reset_index(name="Accuracy")
    .sort_values("Label")
)

plt.rcParams["font.sans-serif"] = ["Microsoft JhengHei"]
plt.figure(figsize=(18,6))
plt.bar(label_acc["Label"], label_acc["Accuracy"])
plt.ylim(0,1)
plt.xlabel("題目標籤"); plt.ylabel("正確率"); plt.title("各題目正確率")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()
